In [5]:
# This program obtains timestamps from our videos marking when asterisks appear on our ATM
# @author: Denisolt Shakhbulatov, Kendall Molas, Tristan Gurtler
from PIL import Image
import sys
import pandas as pd
import os
import csv

In [73]:
###
# Input the file names
# @input startTime - Start video from a certain time-frame, empirically determined
# @input videoFileName - Input video file name
# @input imageFileSaveArea - Folder where extracted images will be saved
# @input frameFileName - CSV file that will contain timestamp data
# @input contrastVal - contrast at which images will be changed to, determined empirically
# @input brightnessVal - brightness at which images will be adjusted to due to video being too dark - determined empirically
# @input cropImage - [x,y,z,w] | [x,y] is the image size, and [z,w] is the coordinate on the image where it will start creating
# the size image
###

startTime = ''
videoFileName = ''
videoFileExtension = '.MP4'
videoToBeProcessed = videoFileName + videoFileExtension
imageFileSaveArea = 'images' + videoFileName
frameFileName = 'frames' + videoFileName + '.csv'
contrastVal = '50'
brightnessVal = '130.00'
cropImage = ''

In [74]:
# Creates directory for all the images of the video to be saved, as well as create a csv file with the time_stamp data.
try:
    os.system("mkdir ../Results/Image/" + imageFileSaveArea)
    
    # Use this for standard processing (the lighting of the video is not too dim for the contrast to affect the capturing the images dramatically)
    os.system("ffmpeg -ss " + startTime + " -i ../Videos/" + videoToBeProcessed + " -an -vf crop=" + cropImage + ",eq=contrast=" + contrastVal + " ../Results/Image/" + imageFileSaveArea + "/%06d.png")
    
    # If video is too dim and images appear to be noisy, use this option to add the brightness as well.
    #os.system("ffmpeg -ss " + startTime + " -i ../Videos/" + videoToBeProcessed + " -an -vf crop=" + cropImage + ",eq=contrast=" + contrastVal + ":brightness=" + brightnessVal + " ../Results/Image/" + imageFileSaveArea + "/%06d.png")
    
    # Debugging purposes
    #os.system("ffmpeg -ss " + startTime + " -i ../Videos/" + videoToBeProcessed + " -an -vf crop=" + cropImage + " ../Results/Image/" + imageFileSaveArea + "/%06d.png")
    
    #os.system("ffprobe -f lavfi -i movie=../Videos/" + videoToBeProcessed + " -show_frames -show_entries frame=pkt_pts_time -of csv=p=0 > ../Results/" + frameFileName)
    print 'Successful'
except:
    print 'ERROR'

Successful


In [29]:
##
# This function checks if the image we are looking at is actually a PIN entry screen at all
#
# @input image - the image we want to check for being a PIN Entry screen
# @returns whether or not we are in a PIN Entry screen

def determine_background(i,k):
    val = rgb_im.getpixel((i, k))
    # values inside getpixel are the px of the location of each symbol
    if(val == (255,255,255)):
        return False
    else:
        return True

In [30]:
##
# This function checks at specified locations to determine if an asterisk is present there
#
# @input image - the image we want to check if an asterisk has appeared inside of
# @input coord_topleft - the top left corner of the box we are checking
# @input coord_bottomright - the bottom right corner of the box we are checking
# @input threshold - what percentage of pixels should be black to determine that an asterisk has appeared
# @returns whether or not an asterisk has appeared at the specified location

def determine_hash(image, coord_topleft, coord_bottomright, threshold=0.005):
    total_pixels = 0
    num_black_pixels = 0

    # range through our box and check for black pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if is_color_black(r, g, b):
                num_black_pixels += 1
            total_pixels += 1

    return (float(num_black_pixels) / total_pixels) >= threshold

In [31]:
# This function checks if a given color (by rgb values) is black or gray
def is_color_black(r, g, b):
    flag_color_is_grayscale = r == g and g == b
    flag_color_is_dark = r <= 211 and g <= 211 and b <= 211
    
    return flag_color_is_grayscale and flag_color_is_dark

In [32]:
# Initialize data frame where the values of the asterisks will be stored
df = pd.DataFrame()
# Begin of time_stamp
time_stamp = 0
# Match the image numbers
frame = 1
indir = '../' + imageFileSaveArea

##
# Locations for scanning asterisk in the area before the shift occurs
##
first_asterisk_area_before_top_left = (,)
first_asterisk_area_before_bottom_right = (,)

second_asterisk_area_before_top_left = (,)
second_asterisk_area_before_bottom_right =  (,)

third_asterisk_area_before_top_left = (,)
third_asterisk_area_before_bottom_right = (,)

fourth_asterisk_area_before_top_left = (,)
fourth_asterisk_area_before_bottom_right = (,)

##
# Locations for scanning asterisk in the other area, after the shift
##
first_asterisk_area_after_top_left = (,)
first_asterisk_area_after_bottom_right = (,)

second_asterisk_area_after_top_left = (,)
second_asterisk_area_after_bottom_right =  (,)

third_asterisk_area_after_top_left = (,)
third_asterisk_area_after_bottom_right = (,)

fourth_asterisk_area_after_top_left = (,)
fourth_asterisk_area_after_bottom_right = (,)

##
# Read through the files in the sorted directory and determine whether hash has appeared or not
##
for root, dirs, filenames in os.walk(indir):
    for f in sorted(os.listdir(indir)):
        log = open(os.path.join(root, f), 'r')
        im = Image.open(log)
        pix = im.load()
        rgb_im = im.convert('RGB')    
        first_asterisk = determine_hash(rgb_im, first_asterisk_area_before_top_left,first_asterisk_area_before_bottom_right) or determine_hash(rgb_im,first_asterisk_area_after_top_left, first_asterisk_area_after_bottom_right)
        second_asterisk = determine_hash(rgb_im, second_asterisk_area_before_top_left,second_asterisk_area_before_bottom_right) or determine_hash(rgb_im,second_asterisk_area_after_top_left, second_asterisk_area_after_bottom_right)
        third_asterisk = determine_hash(rgb_im, third_asterisk_area_before_top_left,third_asterisk_area_before_bottom_right) or determine_hash(rgb_im,third_asterisk_area_after_top_left, third_asterisk_area_after_bottom_right)
        fourth_asterisk = determine_hash(rgb_im, fourth_asterisk_area_before_top_left,fourth_asterisk_area_before_bottom_right) or determine_hash(rgb_im,fourth_asterisk_area_after_top_left, fourth_asterisk_area_after_bottom_right)
        
        ## 
        # @input background - coordinates are empirically determined
        ##
        
        background = determine_background(34,24)
        
        ##
        # Cause asterisks to return empty if False
        ##
        if (first_asterisk == False):
            first_asterisk = ''
        if (second_asterisk == False):
            second_asterisk = ''
        if (third_asterisk == False):
            third_asterisk = ''
        if (fourth_asterisk == False):
            fourth_asterisk = ''
        
        ##
        # Checks if background is black or white
        # If False - Asterisks will pop up with True/False values
        # Else - Columns will be filled with NaNs
        
        if(background == False):
            df = df.append(pd.DataFrame({'Background': background, 'First': first_asterisk, 'Second': second_asterisk, 'Third': third_asterisk, 'Fourth': fourth_asterisk, }, index=[frame]), ignore_index=False)
        else:
            df = df.append(pd.DataFrame({'Background': background, 'First': 'NaN', 'Second': 'NaN', 'Third': 'NaN', 'Fourth': 'NaN', }, index=[frame]), ignore_index=False)
        frame = frame + 1

In [33]:
# Reads from frames2.csv file and combines to have timestamp and dataframe together
Time = pd.read_csv("../" + frameFileName, 
                  names = ["Time_stamp"])
result = pd.concat([df, Time], axis=1, join='inner')

outputFileForUser = 'results' + videoFileName + '.csv'

result = result.reindex(columns=['Time_stamp','Background', 'First', 'Second', 'Third', 'Fourth']).to_csv('../' + outputFileForUser, index=True)
result

IOError: File ../frames110a.csv does not exist

In [ ]:
newReader = pd.read_csv('../' + outputFileForUser)

# Boolean to find the first keypress and prevent from printing out every single line of the csv file to output

firstKP = False
secondKP = False
thirdKP = False
fourthKP = False
clr = False

with open('../relevantKeyPresses' + videoFileName + '.txt', 'w') as textOutput:
    # Iterate through every row in the csv file and check specific columns for True values
    for index, row in newReader.iterrows():
        ## First Asterisk Occurance
        # Checks first column for True
        if (newReader.First.iloc[index] == True and pd.isnull(newReader['Second'].iloc[index]) == True
           and pd.isnull(newReader['Third'].iloc[index]) == True and pd.isnull(newReader['Fourth'].iloc[index]) == True):

           # Check to see if a first keypress was already found
            if (firstKP == False):
                if (clr == False):
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    firstKP = True
                else:
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    clr = False

            # If a first keypress was already found, and becomes missing due to a CLEAR, reset boolean
            elif (firstKP == True and pd.isnull(newReader['First'].iloc[index]) == True):
                textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                firstKP = False
                clr = True

        ## Second Asterisk Occurance
        # Check second column for True
        elif (newReader.Second.iloc[index] == True and pd.isnull(newReader['Third'].iloc[index]) == True
             and pd.isnull(newReader['Fourth'].iloc[index]) == True):
            # Check to see if the second keypress was already found
            if (secondKP == False):
                if (clr == False):
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    secondKP = True
                else:
                    clr = False
            # If a second keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (secondKP == True and pd.isnull(newReader['Second'].iloc[index]) == True):
                textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                secondKP = False
                clr = True

        ## Third Asterisk Occurance
        # Check third column for True
        elif (newReader.Third.iloc[index] == True and thirdKP == False and pd.isnull(newReader['Fourth'].iloc[index]) == True):
            # Check to see if the third keypress was already found
            if (thirdKP == False):
                if (clr == False):    
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    thirdKP = True
                else:
                    clr = True
            # If a third keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (thirdKP == True and pd.isnull(newReader['Fourth'].iloc[index]) == True):
                textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                thirdKP = False
                clr = True

        ## Fourth Asterisk Occurance
        # Check fourth column for True
        # Check to see if fourth keypress was found
        elif (newReader.Fourth.iloc[index] == True and fourthKP == False and pd.isnull(newReader['First'].iloc[index]) == False and
              pd.isnull(newReader['Second'].iloc[index]) == False and pd.isnull(newReader['Third'].iloc[index]) == False):
                if (clr == False):
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    fourthKP = True
                else:
                    clr = False

        # Check to see if a key was removed
        elif (pd.isnull(newReader['Fourth'].iloc[index]) == True and fourthKP == True):
            # Once all keys were found and the screen clears, reset all booleans
            if (pd.isnull(newReader['First'].iloc[index]) == pd.isnull(newReader['Second'].iloc[index]) == 
             pd.isnull(newReader['Third'].iloc[index]) == True):
                # Check next row to see if there is a null val
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    clr = False
                    textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    textOutput.write('---------------------------------------\n')
            elif (fourthKP == True):
                for x in range (1,3):
                    ## Catch any small mistakes
                    # Due to high frame rate capture, it sometimes captures images where some of the asterisks do not disappear at the
                    # same time when area is cleared.
                    if (pd.isnull(newReader['First'].iloc[index+x]) != pd.isnull(newReader['Second'].iloc[index+x]) != 
                         pd.isnull(newReader['Third'].iloc[index+x]) != pd.isnull(newReader['Fourth'].iloc[index+x]) == True):
                        textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
            # Detect if the fourth asterisk has been cleared    
            else:
                fourthKP = False
                clr = True

with open('../relevantKeyPresses' + videoFileName + '.txt', 'r') as input:
    with open('../cleanedRelevantKeyPresses' + videoFileName + '.txt','wb') as output: 
        for line in input:
            if (line!="Name: Time_stamp, dtype: float64"+"\n"):
                output.write(line)
print 'Complete'